# LeetCode 99: Recover Binary Search Tree

## 1. 問題の分析

### 競技プログラミング視点での分析
- **核心**: BSTの中順走査（in-order traversal）は昇順ソート列を生成する
- **2ノード入れ替えの影響**:
  - 隣接ノードの入れ替え → 1箇所の違反（prev > curr）
  - 非隣接ノードの入れ替え → 2箇所の違反
- **最適解**: Morris Traversalで O(1) 空間を実現可能

### 業務開発視点での分析
- **入力検証**: ノード数2以上保証、null検証
- **副作用**: 値のswapのみ、構造変更なし
- **エラー処理**: 不正なツリー構造に対する防御

### JavaScript特有の考慮点
- **オブジェクト参照**: ノード参照を保持し、最後にswap
- **プロパティアクセス**: `.val`, `.left`, `.right` の安定的アクセス
- **Morris走査**: 一時的なポインタ変更のみ、追加オブジェクト生成なし

---

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | JS実装コスト | 可読性 | 備考 |
|-----------|-----------|-----------|-------------|--------|------|
| 方法A: 配列保存 | O(n) | O(n) | 低 | 高 | 全ノード配列化後にソート比較 |
| 方法B: 再帰走査 | O(n) | O(h) | 低 | 高 | コールスタック使用 |
| 方法C: Morris走査 | O(n) | O(1) | 中 | 中 | スレッディングで空間最適 |

---

## 3. 選択したアルゴリズムと理由

- **選択したアプローチ**: 方法C（Morris Traversal）
- **理由**:
  - Follow-upでO(1)空間が要求されている
  - ツリー構造を一時的に変更するが最終的に復元
  - 追加メモリ割り当てなし
- **JavaScript特有の最適化ポイント**:
  - while ループによる反復処理（再帰スタック回避）
  - 参照のみ保持、新規オブジェクト生成なし
  - 最小限の変数宣言

---

## 4. コード実装（solution.js）---

```js
'use strict';

/**
 * Definition for a binary tree node.
 */
class TreeNode {
  /**
   * @param {number} [val=0]
   * @param {TreeNode|null} [left=null]
   * @param {TreeNode|null} [right=null]
   */
  constructor(val = 0, left = null, right = null) {
    this.val = val;
    this.left = left;
    this.right = right;
  }
}

/**
 * Recovers a BST where exactly two nodes were swapped by mistake.
 * Uses Morris In-order Traversal for O(1) space complexity.
 * 
 * @param {TreeNode} root - Root of the BST
 * @returns {void} Modifies tree in-place
 * @throws {TypeError} If root is not a TreeNode
 * @throws {RangeError} If tree has fewer than 2 nodes
 * 
 * Time: O(n) - visits each node at most twice
 * Space: O(1) - only uses constant extra pointers
 */
function recoverTree(root) {
  // Input validation
  if (root === null || !(root instanceof TreeNode)) {
    throw new TypeError('Root must be a valid TreeNode');
  }

  let first = null;   // First swapped node (larger in first violation)
  let second = null;  // Second swapped node (smaller in violation)
  let prev = null;    // Previous node in in-order sequence
  let curr = root;

  // Morris In-order Traversal
  while (curr !== null) {
    if (curr.left === null) {
      // No left subtree: process current, move right
      checkViolation();
      prev = curr;
      curr = curr.right;
    } else {
      // Find in-order predecessor (rightmost in left subtree)
      let predecessor = curr.left;
      while (predecessor.right !== null && predecessor.right !== curr) {
        predecessor = predecessor.right;
      }

      if (predecessor.right === null) {
        // Create thread: link predecessor to current
        predecessor.right = curr;
        curr = curr.left;
      } else {
        // Thread exists: remove it, process current
        predecessor.right = null;
        checkViolation();
        prev = curr;
        curr = curr.right;
      }
    }
  }

  // Swap values to recover BST
  if (first !== null && second !== null) {
    const temp = first.val;
    first.val = second.val;
    second.val = temp;
  }

  /**
   * Check for BST violation at current position
   * Updates first/second pointers when violation found
   */
  function checkViolation() {
    if (prev !== null && prev.val > curr.val) {
      if (first === null) {
        // First violation: prev is the first swapped node
        first = prev;
      }
      // Always update second (handles both adjacent and non-adjacent cases)
      second = curr;
    }
  }
}

// ============ LeetCode Submission Format ============
/**
 * @param {TreeNode} root
 * @return {void} Do not return anything, modify root in-place instead.
 */
var recoverTreeLeetCode = function(root) {
  let first = null, second = null, prev = null, curr = root;

  while (curr) {
    if (!curr.left) {
      check();
      prev = curr;
      curr = curr.right;
    } else {
      let pred = curr.left;
      while (pred.right && pred.right !== curr) pred = pred.right;

      if (!pred.right) {
        pred.right = curr;
        curr = curr.left;
      } else {
        pred.right = null;
        check();
        prev = curr;
        curr = curr.right;
      }
    }
  }

  if (first && second) [first.val, second.val] = [second.val, first.val];

  function check() {
    if (prev && prev.val > curr.val) {
      if (!first) first = prev;
      second = curr;
    }
  }
};

module.exports = { TreeNode, recoverTree, recoverTreeLeetCode };
```
Analyze Complexity
Runtime 3 ms
Beats 66.94%
Memory 65.44 MB
Beats 42.15%

## 5. 追加メモ（アルゴリズム解説）

### Morris Traversal の動作原理

```
     4
    / \
   2   6
  / \
 1   3
```

1. **スレッディング**: 左部分木の最右ノードから現在ノードへの一時リンクを作成
2. **復元**: 2回目の訪問時にリンクを削除し、ノードを処理
3. **利点**: スタック/再帰なしで中順走査を実現

### 違反検出のロジック

| ケース | 例（正しい順序 → 入れ替え後） | 違反箇所 |
|--------|------------------------------|----------|
| 隣接swap | [1,2,3,4] → [1,3,2,4] | (3,2) 1回 |
| 非隣接swap | [1,2,3,4] → [1,4,3,2] | (4,3), (3,2) 2回 |

- **first**: 最初の違反の「大きい方」（prev）
- **second**: 最後の違反の「小さい方」（curr）

### JS最適化チェックリスト
- ✅ while ループ使用（再帰回避）
- ✅ オブジェクト生成なし（参照のみ）
- ✅ 分割代入でswap（LeetCode版）
- ✅ 早期検証でホットパス保護
- ✅ クロージャは1つのみ（checkViolation）

# 分析と最適化版

## パフォーマンス低下の原因分析

| 問題点 | 影響 | 改善策 |
|--------|------|--------|
| クロージャ `checkViolation` | 関数呼び出しオーバーヘッド | インライン化 |
| 分割代入 `[a, b] = [b, a]` | 一時配列生成 | 直接swap |
| 制約が小さい（n≤1000） | Morris不要な可能性 | 再帰版も検討 |
| 冗長なnullチェック | 微小だが累積 | 簡略化 |

## 最適化版コード
## 推奨する提出順序

```js
'use strict';

/**
 * 最適化版1: Morris Traversal（O(1)空間・インライン化）
 * @param {TreeNode} root
 * @return {void}
 */
var recoverTree = function(root) {
  let first = null, second = null, prev = null, curr = root, pred;
  
  while (curr) {
    if (curr.left) {
      // Find predecessor
      pred = curr.left;
      while (pred.right && pred.right !== curr) pred = pred.right;
      
      if (pred.right) {
        // Remove thread, process node
        pred.right = null;
        if (prev && prev.val > curr.val) {
          if (!first) first = prev;
          second = curr;
        }
        prev = curr;
        curr = curr.right;
      } else {
        // Create thread
        pred.right = curr;
        curr = curr.left;
      }
    } else {
      // No left child, process node
      if (prev && prev.val > curr.val) {
        if (!first) first = prev;
        second = curr;
      }
      prev = curr;
      curr = curr.right;
    }
  }
  
  // Direct swap (no destructuring)
  const temp = first.val;
  first.val = second.val;
  second.val = temp;
};

Analyze Complexity
Runtime 2 ms 
Beats 94.1%
Memory 65.30 MB
Beats 58.68%

/**
 * 最適化版2: 再帰版（制約n≤1000では実用的に最速）
 * 空間O(h)だがV8最適化が効きやすい
 * @param {TreeNode} root
 * @return {void}
 */
var recoverTreeRecursive = function(root) {
  let first = null, second = null, prev = null;
  
  (function dfs(node) {
    if (!node) return;
    dfs(node.left);
    if (prev && prev.val > node.val) {
      if (!first) first = prev;
      second = node;
    }
    prev = node;
    dfs(node.right);
  })(root);
  
  const temp = first.val;
  first.val = second.val;
  second.val = temp;
};

Analyze Complexity
Runtime 3 ms
Beats 66.94%
Memory 65.14 MB
Beats 64.05%

/**
 * 最適化版3: 超軽量版（最小コード）
 * @param {TreeNode} root
 * @return {void}
 */
var recoverTreeMinimal = function(root) {
  let f, s, p;
  const go = n => {
    if (!n) return;
    go(n.left);
    if (p && p.val > n.val) { if (!f) f = p; s = n; }
    p = n;
    go(n.right);
  };
  go(root);
  const t = f.val; f.val = s.val; s.val = t;
};

module.exports = { recoverTree, recoverTreeRecursive, recoverTreeMinimal };

Analyze Complexity
Runtime 5 ms
Beats 23.97%
Memory 65.18 MB
Beats 64.05%

```

### 🥇 まず試す: 再帰版（最適化版2 or 3）

```javascript
var recoverTree = function(root) {
  let first = null, second = null, prev = null;
  
  (function dfs(node) {
    if (!node) return;
    dfs(node.left);
    if (prev && prev.val > node.val) {
      if (!first) first = prev;
      second = node;
    }
    prev = node;
    dfs(node.right);
  })(root);
  
  const temp = first.val;
  first.val = second.val;
  second.val = temp;
};
```

**理由**:
- 制約 `n ≤ 1000` ではスタックオーバーフローの心配なし
- V8は再帰をインライン展開しやすい
- 関数呼び出しが単純で最適化されやすい

### 🥈 次に試す: Morris最適化版（最適化版1）

Morris版でもクロージャを排除したので改善されているはずです。

---

## 改善ポイント詳細

| 変更点 | Before | After | 効果 |
|--------|--------|-------|------|
| checkViolation | 関数呼び出し | インライン | 呼び出しコスト削減 |
| swap方式 | `[a,b]=[b,a]` | `temp`使用 | 一時配列生成回避 |
| 変数宣言 | 複数行 | 1行にまとめ | 微小な最適化 |
| null比較 | `!== null` | `!node` | 簡潔（効果は微小） |

---

## 期待される改善

| 指標 | 現在 | 期待値 |
|------|------|--------|
| Runtime | 3ms (66.94%) | 0-2ms (85-95%) |
| Memory | 65.44MB (42.15%) | 55-60MB (60-80%) |

再帰版で試してみてください。LeetCodeの環境では再帰版の方が速いことが多いです。